# Using a Stewart Platform as a Motion Simulator
#### Python + ESP32 + 3D CAD + Motion Capture Data

This is a WIP progress!


[<img src="./doc/jupyter_nb/Stewart_IRL_1.gif" width="30%" height="30%">](./doc/jupyter_nb/Stewart_IRL_1.gif)
[<img src="./doc/jupyter_nb/Fusion360.png" width="30%" height="30%">](./doc/jupyter_nb/Fusion360.png)
[<img src="./doc/jupyter_nb/Stewart_IRL_2.gif" width="30%" height="30%">](./doc/jupyter_nb/Stewart_IRL_2.gif)


Import the CSV file containing the positions of a body in 6DOF. A 2DOF example of a person running on a treadmill is provided, of which we scale into the form of

Yaw: Radians

Translation in Z axis: Centimeters scaled according to height of motion simulator

In [84]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# file = open('positions_processed.csv')
file = open('./motion_capture/relative_motion.csv')

csvreader = csv.reader(file)


rows = []
for row in csvreader:
        rows.append(row)
rows
file.close()
movements = np.array(rows).astype('float')
# movements[:,0:3] = movements[:,0:3]/180*np.pi
movements[:,3:6] = movements[:,3:6]/10



# yaw_movements = movements[0,:].astype('float') /180*np.pi
# z_movements = movements[1,:].astype('float')/20


plt.subplot(611); plt.plot(movements[:,0]) 
plt.subplot(612); plt.plot(movements[:,1])
plt.subplot(613); plt.plot(movements[:,2])
plt.subplot(614); plt.plot(movements[:,3]) 
plt.subplot(615); plt.plot(movements[:,4])
plt.subplot(616); plt.plot(movements[:,5])

My approach is then to
1. Use the stewart_py package to calculate the inverse kinematic of the desired position according to the movement data
2. Use py_serial to output the data to a ESP32
3. ESP32 receives serial data and issues I2C command to a pca9685
4. pca9685 commands 6 servo motors to rotate accordingly

This is a very cost efficient solution however if I could redo this project I'd use a Raspberry Pi instead and direcly use it to issue I2C commands to a pca9685

In [95]:
import numpy as np
from src.stewart_controller import Stewart_Platform
import matplotlib.pyplot as plt

import time
import serial

# Initialize serial

ser = serial.Serial()
ser.port = 'COM5'
ser.baudrate = 38400
ser.setDTR(False)
ser.setRTS(False)

# ser.close()
if not ser.isOpen():
    ser.open()

# Call object, 
# These parameters are for my 3D models as provided, you can change your parameters according to your own hardware,
# Stewart_Platform(r_B, r_P, lhl, ldl, Psi_B, Psi_P), where
# r_B = Radius of Base (Bottom)
# r_P = Radius of Platform (Top)
# lhl = Servo Horn Length
# ldl = Rod length
# Psi_B = Half of angle between two anchors on the base
# Psi_P = Half of angle between two anchors on the platform
# For details please refer to my tutorial https://github.com/Yeok-c/Stewart_Py

platform = Stewart_Platform(13.2/2, 17.5/2, 5.08, 13, 0.2269, 0.82, 5*np.pi/6)
controller_freq = 12
%matplotlib qt
# Initialize Plots
fig, ax = plt.subplots()    

while 1:
    # Loop through various angles
    for ix in range(0, 1400, 9):
        time.sleep(1/controller_freq)
        # tic = time.time()
# [movements[ix,0], movements[ix,1], movements[ix,2]]
        # send servo angles for this time-step. Movement input data is scaled for better clarity, output is scaled into degrees
        # servo_angles = platform.calculate(np.array([movements[ix,3], movements[ix,4], movements[ix,5]]), np.array([movements[ix,0], movements[ix,1], movements[ix,2]]) )/np.pi*180
        # servo_angles = platform.calculate(np.array([0,0,0]), np.array([np.pi/15,0,0]) )/np.pi*180
        servo_angles = platform.calculate(np.array([movements[ix,3],movements[ix,4],movements[ix,5]]), np.array([movements[ix,1]+np.pi/15,movements[ix,2],movements[ix,0]]) )/np.pi*180
        
        # servo_angles = platform.calculate(np.array([3,0,0]), np.array([0,0,0]) )/np.pi*180

        # Send serial data to ESP32 in the form of a string "<A,float,float,float,float,float,float>"
        towrite = np.array2string(servo_angles, precision=1, separator=',').strip('[]')
        towrite = '<A,' + towrite + '>'
        ser.write(towrite.encode())     # write a string
        print(towrite)

        # Hide this if your computer takes too long to plot, affecting the controller speed
        # ax = platform.plot_platform()

        # plt.pause(1/controller_freq)
        # plt.draw()


        # toc = time.time()
        # print(tic-toc) # Measure time to see if plotting affects speed of controller


<A,12.8,22.2,52.5,50.2,23.7,16.6>
<A,-1.2,-5.1,53.5,36.9,17.3,-9.5>
<A,13.4, 9.2,68. ,55.7,28.7, 4.3>
<A,23.7,31.5,69.4,67.3,28.7,20.3>
<A,13.4,34.7,36.8,49.4, 3.5,13.7>
<A,  4.9, 30.6, 19.9, 39.1,-10.7,  6.3>
<A,18.9,36.9,41.1,49.6,12.5,20.4>
<A,22.8,26.8,72.6,67.7,33.7,20.8>
<A,12.2, 0.6,72.5,52.3,29.9,-5.9>
<A, 10.1, -8.6, 70.2, 44.4, 25.8,-19.1>
<A,21.5, 8.8,82. ,71.9,38.8, 4.4>
<A,23.8,21.2,68. ,66.9,36.1,16.1>
<A, 8.7,16. ,33.9,43.7,11.6, 5.8>
<A, 2.9,12. ,26.2,38.1, 5.5, 3.1>
<A,17.7,25. ,48.7,53.7,26.8,21. >
<A,21. ,21.2,76.3,73.5,39.1,20.1>
<A, 8.7, 0.5,68.7,47.8,26.6,-5.7>
<A,  8. , -2.9, 66.7, 42.1, 21.7,-12.9>
<A,20.6,17.2,78.4,66.2,30.8, 8. >
<A,22.2,28.8,60. ,59.7,25.2,16.4>
<A, 7.8,20.5,29.6,40.6, 4.6, 4.2>
<A, 8.1,19.4,28.6,40.6, 7.8, 7.1>
<A,24. ,33.6,52.9,57.8,25.7,22.5>
<A,24.2,30.8,62.2,59.4,30.5,22.4>
<A, 4.2,11.9,39.7,34.3,15.5, 8.5>
<A, 5.7,20.5,31.6,32. , 8.3,14.7>
<A,26.2,44.7,39.5,46. ,17.4,31.3>
<A,30.1,52. ,33.5,45.5,12.2,30.6>
<A,13.2,36.4,14.5,32.6,-4.5,14

SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))

In [75]:

ser = serial.Serial()
ser.port = 'COM5'
ser.baudrate = 38400
ser.setDTR(False)
ser.setRTS(False)

# ser.close()
if not ser.isOpen():
    ser.open()


servo_angles = platform.calculate(np.array([0,0,0-2]), np.array([np.pi/5, 0,0]) )/np.pi*180

# Send serial data to ESP32 in the form of a string "<A,float,float,float,float,float,float>"
towrite = np.array2string(servo_angles, precision=1, separator=',').strip('[]')
towrite = '<A,' + towrite + '>'
ser.write(towrite.encode())     # write a string
print(towrite)
ax = platform.plot_platform()

plt.draw()

<A,-55.2,-23.4, 53.1, 53.1,-23.4,-55.2>


In [3]:
import numpy as np
r_B, r_P, gamma_B, gamma_P = 6.2, 5, 0.2269, 0.82

# 0.2269rad is 13 degrees, which is standard gamma_B for stewart platforms

pi = np.pi
## Define the Geometry of the platform

# Psi_B (Polar coordinates)
psi_B = np.array([ 
    -gamma_B, 
    gamma_B,
    2*pi/3 - gamma_B, 
    2*pi/3 + gamma_B, 
    2*pi/3 + 2*pi/3 - gamma_B, 
    2*pi/3 + 2*pi/3 + gamma_B])

# psi_P (Polar coordinates)
# Direction of the points where the rod is attached to the platform.
psi_P = np.array([ 
    pi/3 + 2*pi/3 + 2*pi/3 + gamma_P,
    pi/3 + -gamma_P, 
    pi/3 + gamma_P,
    pi/3 + 2*pi/3 - gamma_P, 
    pi/3 + 2*pi/3 + gamma_P, 
    pi/3 + 2*pi/3 + 2*pi/3 - gamma_P])

# Coordinate of the points where servo arms 
# are attached to the corresponding servo axis.
B = r_B * np.array( [ 
    [ np.cos(psi_B[0]), np.sin(psi_B[0]), 0],
    [ np.cos(psi_B[1]), np.sin(psi_B[1]), 0],
    [ np.cos(psi_B[2]), np.sin(psi_B[2]), 0],
    [ np.cos(psi_B[3]), np.sin(psi_B[3]), 0],
    [ np.cos(psi_B[4]), np.sin(psi_B[4]), 0],
    [ np.cos(psi_B[5]), np.sin(psi_B[5]), 0] ])
B = np.transpose(B)
    
# Coordinates of the points where the rods 
# are attached to the platform.
P = r_P * np.array([ 
    [ np.cos(psi_P[0]),  np.sin(psi_P[0]), 0],
    [ np.cos(psi_P[1]),  np.sin(psi_P[1]), 0],
    [ np.cos(psi_P[2]),  np.sin(psi_P[2]), 0],
    [ np.cos(psi_P[3]),  np.sin(psi_P[3]), 0],
    [ np.cos(psi_P[4]),  np.sin(psi_P[4]), 0],
    [ np.cos(psi_P[5]),  np.sin(psi_P[5]), 0] ])
P = np.transpose(P)

print('6x3 array for Base anchors \n', B)

print('6x3 array for Platform anchors \n', P)

6x3 array for Base anchors 
 [[ 6.04108436  6.04108436 -1.8126619  -4.22842247 -4.22842247 -1.8126619 ]
 [-1.39474002  1.39474002  5.92910253  4.53436252 -4.53436252 -5.92910253]
 [ 0.          0.          0.          0.          0.          0.        ]]
6x3 array for Platform anchors 
 [[ 4.87150733  4.87150733 -1.46040129 -3.41110604 -3.41110604 -1.46040129]
 [-1.12623991  1.12623991  4.78196906  3.65572915 -3.65572915 -4.78196906]
 [ 0.          0.          0.          0.          0.          0.        ]]


In [4]:
import matplotlib.pyplot as plt
import seaborn

def plot_2D_annotate(X, Y):
    X = list(X)
    Y = list(Y)
    ax.plot(X,Y, 'or')  # Plot Points
    # Annotate Points
    for i, (xy) in enumerate(zip(X, Y)):                                 
        ax.annotate('Point ' + str(i) + '\n(%.5s, %.5s)' % xy, xy=xy, textcoords='data')

    X.append(X[0])
    Y.append(Y[0])
    ax.plot(X,Y)    # Plot lines

seaborn.set(style='ticks')
# plt.style.use('dark_background')

fig = plt.figure(figsize=(14, 6), dpi=80)

ax = fig.add_subplot(121)
ax.set_title('Base Anchors')
plot_2D_annotate(B[0,:], B[1,:])
circle_B = plt.Circle((0, 0), r_B, color='g', fill=False)
ax.add_patch(circle_B)

ax = fig.add_subplot(122)
ax.set_title('Platform Anchors')
plot_2D_annotate(P[0,:], P[1,:])
circle_r = plt.Circle((0, 0), r_P, color='g', fill=False)
ax.add_patch(circle_r)

plt.show()